In [465]:
import pandas as pd

df = pd.read_csv('food_list.csv',encoding='cp949')
data = df[['구분', '음식명']] # 필요한 데이터만 가져오기

for i in range (len(data)): # ,가 포함되거나 한 글자인 카테고리명 변경 / 문자가 있거나 한 글자인 음식명 변경
	if (',' in data['구분'][i]):
		data.loc[i, "구분"] = data['구분'][i].replace(',', '_')

	if (len(data['구분'][i]) == 1):
		data.loc[i, "구분"] = data['구분'][i] + "_"

	if (len(data['음식명'][i]) == 1):
		data.loc[i, "음식명"] = data['음식명'][i] + "_"

	if (" / " in data['음식명'][i]):
		data.loc[i, "음식명"] = data['음식명'][i].replace(" / ", '_')
	
	if ("/" in data['음식명'][i]):
		data.loc[i, "음식명"] = data['음식명'][i].replace("/", '_')

/var/folders/7y/c05dgdf51dl2hthqhmsb1pf80000gn/T/ipykernel_6009/3860020494.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.loc[i, "구분"] = data['구분'][i] + "_"
/var/folders/7y/c05dgdf51dl2hthqhmsb1pf80000gn/T/ipykernel_6009/3860020494.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.loc[i, "구분"] = data['구분'][i].replace(',', '_')
/var/folders/7y/c05dgdf51dl2hthqhmsb1pf80000gn/T/ipykernel_6009/3860020494.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.ht

In [466]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(ngram_range=(1,2)) 
cv_category = cv.fit_transform(data['구분'])
cv.vocabulary_ # 카테고리별 인덱스 번호

cv.vocabulary_

{'면_': 19,
 '떡_': 15,
 '찜_': 48,
 '어류': 33,
 '구이': 6,
 '젓갈': 42,
 '조림': 43,
 '채소류': 49,
 '나물': 11,
 '선_': 27,
 '전_적': 39,
 '볶음': 24,
 '양념': 32,
 '회_': 57,
 '국_탕': 7,
 '과일류': 5,
 '서류': 26,
 '김치': 10,
 '밥_': 22,
 '죽_': 45,
 '한과': 55,
 '마른찬': 17,
 '갑각류': 0,
 '생채': 25,
 '음청류': 36,
 '고명': 2,
 '장아찌': 37,
 '육류': 35,
 '전골': 41,
 '청주': 50,
 '패류': 53,
 '해조류': 56,
 '만두': 18,
 '찌개': 47,
 '곡류': 3,
 '연체류': 34,
 '묵_': 21,
 '기타': 9,
 '쌈_': 31,
 '견과류': 1,
 '편육': 54,
 '두부': 14,
 '떡국': 16,
 '숙채': 29,
 '두류': 13,
 '곡류 견과류': 4,
 '증류주': 46,
 '버섯류': 23,
 '탕_': 52,
 '수제비': 28,
 '순대': 30,
 '나물 생채': 12,
 '전_적_튀김': 40,
 '초_': 51,
 '국수': 8,
 '족편': 44,
 '전_': 38,
 '무침': 20}

In [467]:
from sklearn.metrics.pairwise import cosine_similarity
similarity_category = cosine_similarity(cv_category, cv_category)
sim_category = similarity_category.argsort()[:,::-1]
print(sim_category)
sim_category.shape

[[  0  80 709 ... 493 492 375]
 [414 421  26 ... 493 492   0]
 [619 413 257 ... 491 490   0]
 ...
 [750 113  32 ... 488 487   0]
 [414 421  26 ... 493 492   0]
 [750 113  32 ... 488 487   0]]


(751, 751)

In [468]:
cv = CountVectorizer(ngram_range=(1,2)) 
cv_menuname = cv.fit_transform(data['음식명'])
similarity_menuname = cosine_similarity(cv_menuname, cv_menuname)
sim_menuname = similarity_menuname.argsort()[:,::-1]
print(sim_menuname)
sim_menuname.shape

[[  0 246 255 ... 496 495 375]
 [  1 750 246 ... 496 495   0]
 [  2 750 246 ... 496 495   0]
 ...
 [748 750 246 ... 495 494   0]
 [749 246 255 ... 496 495   0]
 [750 246 255 ... 496 495   0]]


(751, 751)

In [469]:
cate_menu_co = (
	+ similarity_category * 0.8
	+ similarity_menuname * 0.2
)

cate_menu_co_idx = cate_menu_co.argsort()[:,::-1]
print(cate_menu_co_idx)

[[  0  80 709 ... 493 492 375]
 [  1 414 314 ... 493 492   0]
 [  2 619 657 ... 491 490   0]
 ...
 [748 750 616 ... 488 487   0]
 [749 414 314 ... 493 492   0]
 [750 113  32 ... 488 487   0]]


In [531]:
import numpy as np

def recommend_menu(df, sorted_idx, menu_name, top=30):
	target_menu_idx = df[df['음식명'] == menu_name].index.values

	sim_idx = sorted_idx[target_menu_idx, :top].reshape(-1)

	result = df.iloc[sim_idx]
	return result

In [532]:
menu_name = "죽" # 입력

if (len(menu_name) == 1):
	menu_name += "_"

recommend_menu(data, cate_menu_co_idx, menu_name)

,구분,음식명
616,죽_,죽_
113,죽_,깨죽
32,죽_,강냉이범벅
710,죽_,합자죽
695,죽_,팥죽
670,죽_,콩죽
291,죽_,미음
641,죽_,채소죽
750,죽_,흰죽
733,죽_,호박죽
